In [55]:
import numpy as np

# A simple numerical example on simulated data

## Simulating needed data

For now, I assume production functions are Cobb-Douglass, households have Cobb-Douglas preferences over final consumption goods, and the matching function is Cobb-Douglas. In principle, we don't need to be so restrictive about the functional forms of production and matching. Instead all of the relevant information about production technologies is captured by the levels and changes of the elasticities outline below. Any two production functions with the same elasticities and changes in elasticities would generate the same responses to shocks.

For each sector $j$, we need the elasticity of production of good $j$ with respect to the intermediate input from each other sector $i$, $\varepsilon^{f_j}_{x_{ji}}$, and the elasticity of production with respect to the labor input $N_j$, $\varepsilon^{f_j}_{N_{j}}$. I sample elasticites of production on the unit simplex using an exponential transformation of the normal distribution. To start, for each industry, draw
\begin{align*}
    \left(z_{j1}, \cdots,z_{jJ},z_{N_j}\right) \sim N\left(\mu,\Sigma\right)
\end{align*}
Then define
\begin{align*}
    \varepsilon^{f_j}_{x_{j1}} = \frac{e^{z_{j1}}}{e^{z_{N_j}}+\sum_{k=1}^{J}e^{z_{jk}}}
\end{align*}
And equivalently for all other sectors. We also need the elasticities of demand with respect to sector $j$'s output, $\varepsilon^{\mathcal{D}}_{c_j}$. Notice that we can treat the final demand as another production sector that uses no labor input. We can sample the demand elasticities using the same procedure outlined above.

Finally, I sample the matching elasticity with respect to vacancies from a uniform distribution on $[0,1]$. 

We let $\bm{\Omega}$ be the matrix of input elasticities for each sector and $\bm{I}-\bm{\Psi}$ be the Leontif inverse.

In [56]:
# Size of network
J = 4 

# Sampling input-output matrix entries 
μ = np.zeros(J+1)
Σ = np.eye(J+1)
z_draws = np.random.multivariate_normal(μ, Σ, J+1)
elasticity_draws = np.exp(z_draws)/np.sum(np.exp(z_draws),1).reshape((J+1,1))

# Elasticities
epsD = elasticity_draws[0,:-1]/np.sum(elasticity_draws[0,:-1])
Omega = elasticity_draws[1:,:-1]
Psi = np.linalg.inv(np.eye(J)-Omega)
epsN = np.diag(elasticity_draws[1:,-1])

# Drawing elasticity of matching function wrt to U
ν = np.random.uniform(size=J)
curlyQ = np.diag(-ν)
curlyF =  np.eye(J) - curlyQ

Finally, we need recruiter producer ratios $\tau_j(\theta_j)$ in each sector. Landais, Michaillat, and Saez (2018) find that the share of recruiters in the US workforce averages around 2.3 percent. I sample recruiter producer ratios uniformly on the interval $[0,0.046]$ to roughly match this fact.

In [57]:
tau = np.random.uniform(low=0,high=0.046,size=J).reshape((J,1))

## Defining shocks
We are interested in the response of sector level and aggregate output and employment to technology shocks $d\log\bm{A}$ and labor force shocks $d\log \bm{H}$. The code below defines the shocks we feed into the model.

In [58]:
# Technology shocks
dlog_A = 0.01*np.ones(J)
dlog_H = 0.01*np.ones(J)

## Defining how wages adjust
Since there are mutual gains from trade once an unemployed worker and a firm meet, wages are not pinned down uniquely in models featuring search and matching frictions in the labor market. We must therefore impose a wage schedule, an assumption about how wages change in response to fundamentals, in order to close the model. 

In particular, we need to specify the elasticity of wages to technology shocks and labor force shocks in each sector, $\left\{\left\{\varepsilon^{w_j}_{A_{ji}},\varepsilon^{w_j}_{H_{ji}}\right\}_{i=1}^{J}\right\}_{j=1}^{J}$. For instance, a simple, but unrealistic, assumption is that wages respond positively to own sector productivity and negatively to own sector labor force, but do not respond to changes in other sectors. In general, we express changes in wages as a function of changes in productivity and the labor force. 
\begin{align}
    d\log \bm{w} &= \bm{\Lambda_{A}} d\log \bm{A} + \bm{\Lambda_{H}} d\log \bm{H} \tag{1}
\end{align}
Where $\bm{\Lambda_{A}}$ contains wage elasticities to productivity changes and $\bm{\Lambda_{H}}$ contains wage elasticities to labor force changes.

In [59]:
# Wage elasticities
epsW_A = np.diag(np.random.uniform(low=0,high=2,size=J))
epsW_H = np.diag(np.random.uniform(low=-2,high=0,size=J))

In [60]:
# Calculating the log change in wages
def WageFunc(dlog_A, dlog_H, epsW_A, epsW_H):
    dlog_w = epsW_A @ dlog_A + epsW_H @ dlog_H
    return dlog_w

In [61]:
#How wages change
dlog_w = WageFunc(dlog_A, dlog_H, epsW_A, epsW_H)

## Tightness propagation
We with wage changes in hand, we solve for first order changes in tightness in terms of $d\log\bm{A}$, $d\log\bm{H}$, and $d\log\bm{w}$. The general formula for changes in tightness, treating sector 1 prices as the numeraire, is
\begin{align}
    d \log \bm{\theta} &=\left(\text{diag}\left(\bm{\mathcal{F}}\right)-\bm{\Xi_{\theta}}\right)^{-1}\left(\left(\bm{I} - \bm{\Psi} \text{diag}\left(\bm{\varepsilon^f_N}\right)\right) \left(d\log \bm{\varepsilon^f_N} + d\log \bm{\lambda} - d\log \bm{H}\right) + \bm{\Xi_{A}} d\log \bm{A}\right) \nonumber \\
    &- \left(\text{diag}\left(\bm{\mathcal{F}}\right)-\bm{\Xi_{\theta}}\right)^{-1} \left(\bm{I}-\bm{\Xi}_{w}\right)d\log \bm{w} \tag{2}
\end{align}
Where 
\begin{align*}
    \bm{\Xi_{A}} &= \begin{bmatrix}
        \Psi_1 \\
        \bm{0} \\
        \vdots \\
        \bm{0}
    \end{bmatrix}, \, \bm{\Xi_\theta} = \begin{bmatrix}
        \Psi_1 \varepsilon^{f_1}_{N_1}\left(\varepsilon^{\mathcal{F}_1}_{\theta_1} + \tau_1(\theta_1)\varepsilon^{\mathcal{Q}_1}_{\theta_1}\right) \\
        \Psi_2 \varepsilon^{f_2}_{N_2} \varepsilon^{\mathcal{F}_2}_{\theta_2} \\
        \vdots \\
        \Psi_J \varepsilon^{f_J}_{N_J} \varepsilon^{\mathcal{F}_J}_{\theta_J}
    \end{bmatrix}, \bm{\Xi_{w}} = \begin{bmatrix}
        \bm{0} \\
        \Psi_2 \varepsilon^{f_2}_{N_2} \\
        \vdots\\
        \Psi_J \varepsilon^{f_J}_{N_J}
    \end{bmatrix}
\end{align*}

In [66]:
def ThetaFunc(dlog_A, dlog_H, dlog_w, dlog_epsN, dlog_lam, Psi, curlyF, curlyQ, epsN, tau):
    
    # Creating matrices
    Xi_a = np.zeros_like(Psi)
    Xi_a[0,:] = Psi[0,:]

    Xi_theta = Psi @ epsN @ curlyF
    Xi_theta[0,:] = Xi_theta[0,:] + Psi[0,:] * epsN[0,0] * tau[0] * curlyQ[0,0]

    Xi_w = Psi @ epsN 
    Xi_w[0,:] = 0

    I = np.eye(Psi.shape[0])

    # Contribution of different components
    Cw = np.linalg.inv(curlyF - Xi_theta) @ (I - Xi_w)
    Ca = np.linalg.inv(curlyF - Xi_theta) @ Xi_a 
    Ch = np.linalg.inv(curlyF - Xi_theta) @ (I - Psi @ epsN)

    # Change in tightness
    dlog_theta = Ch @ (dlog_epsN + dlog_lam - dlog_H) + Ca @ dlog_A - Cw @ dlog_w

    return dlog_theta

Assuming Cobb-Douglas production implies $d\log\bm{\varepsilon^f_N} = d\log \bm{\lambda} = 0$.

In [67]:
dlog_epsN = np.zeros_like(dlog_A)
dlog_lam = np.zeros_like(dlog_A)
dlog_theta = ThetaFunc(dlog_A, dlog_H, dlog_w, dlog_epsN, dlog_lam, Psi, curlyF, curlyQ, epsN, tau)

## Price and output propagation
With changes in tightness in hand, we can now work out how prices and sectoral production changes in response to technology and labor force shocks. Price changes are given by 
\begin{align}
    d \log \bm{p} &=\bm{\Psi} \left(
        \text{diag}\left(\bm{\varepsilon^{f}_{N}}\right) d\log\bm{w} -\text{diag}\left(\bm{\varepsilon^{f}_{N}}\right)\text{diag}\left(\bm{\tau}\right)\text{diag}\left(\bm{\varepsilon^{\mathcal{Q}}_{\theta}}\right)d\log\bm{\theta} - d\log \bm{A} \right) \tag{3}
\end{align}

In [71]:
def PriceFunc():
    return

array([[775.43220389, 738.1006394 , 699.01538381, 944.28586135],
       [740.18842102, 706.95299037, 668.30246554, 902.97542036],
       [451.64090395, 430.48251945, 408.41997481, 550.72350543],
       [766.50936468, 730.86741846, 692.01613526, 936.19067797]])

And output changes are given by
\begin{align}
    d\log \bm{y} &= \bm{\Psi}\left(d\log\bm{A} + \text{diag}\left(\bm{\varepsilon^{f}_{N}}\right)\left(\text{diag}\left(\bm{\mathcal{F}}\right)+\text{diag}\left(\bm{\tau}\right)\text{diag}\left(\bm{\varepsilon^{\mathcal{Q}}_{\theta}}\right)\right)d\log \bm{\theta} + \text{diag}\left(\bm{\varepsilon^{f}_{N}}\right) d\log\bm{H}\right) \nonumber\\
    &-\bm{\Psi} \text{diag}\left(\bm{\varepsilon^{f}_{N}}\right) d\log \bm{\varepsilon^{f}_{N}} + \left(\bm{I} - \bm{\Psi}\text{diag}\left(\bm{\varepsilon^{f}_{N}}\right)\right) d\log \bm{\lambda} \tag{4}
\end{align}


In [ ]:
def OutputFunc():
    return

## Unemployment

## Aggregation